In [1]:
%pip install -qU langchain
%pip install -qU huggingface_hub
%pip install nest-asyncio
%pip install unstructured
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
# Necessary to import OpenAI
with open('assets/openai_api_key', 'r') as f:
    openai_api_key = f.read()
with open('assets/huggingface_api_key', 'r') as f:
    huggingface_api_key = f.read()
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['HUGGINGFACEHUB_API_KEY'] = huggingface_api_key
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from langchain.output_parsers import RegexParser
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import json

# OpenAI QA

We use Langchain to create a Chroma database of our papers against which we will query our questions. The answers are provided in the requested structure, making it straightforward to extract information.

https://python.langchain.com/docs/modules/chains/additional/question_answering.html

https://python.langchain.com/docs/modules/chains/additional/openai_functions_retrieval_qa


Setting up the parameters:

In [ ]:
paper_path = '../data/docs/'
pickle_path = '../data/pickle/'
chroma_path = '../data/chroma/'
MODEL = 'gpt-3.5-turbo'

Creating the vector database:

In [ ]:
def papers_to_vectors(paper_path = paper_path, chroma_path=chroma_path):
    chroma_index = chroma_path + 'index'
    if os.path.exists(chroma_index):
        docsearch = Chroma(persist_directory=chroma_path)
    else:
        loader = DirectoryLoader(paper_path)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)
        texts = text_splitter.split_documents(documents)
        embeddings = OpenAIEmbeddings()
        docsearch = Chroma.from_documents(texts, embeddings,persist_directory='../data/chroma', metadatas=[{"source": str(i)} for i in range(len(texts))])
    docsearch.persist()
    return docsearch

docsearch = papers_to_vectors()



In [ ]:
prompt_template = """Use the journal article as context to fill out the keys to return as defined at the end. You will provide an answer with the format below for the source document I have provided. If you don't know the answer, reply with a blank string (''), do not try to make up an answer. I provide an explanation between brackets of what each key to return is, but do not include those in your answer, just the key name exactly as provided followed by a colon and your answer. 

In addition to answering to the specific keys, also return a 'Score' of how fully it answered the question, the journal article used as context, and the text passage containing the answer. All these keys have to be in the following #format, do not alter the order or skip any key:

Format:
---------
Question: [question here]
Adverse outcome [adverse outcome here]
Bioassay: [bioassay here]
Material diameter measurement: [diameter measurement here]
Numerical value: [mnumerical value of diameter measurement here]
Units: [units here]
Graphene oxide dose: [dose here]
Graphene oxide dose units: [dose units here]
Organism/Biological system: [test subject here]
Score: [score between 0 and 100]
Source: [source, the journal article containing the answer, from documents metadata]
Passage: [the text passage containing the answer]


---------

Context:
---------
{context}
---------
Questions: 
{question}

"""

output_parser = RegexParser(
    regex=r"\nBioassay: (.*)\nMaterial diameter measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",
    output_keys=["Adverse outcome","Bioassay", "Material diameter measurement", "Numerical value", "Units", "Graphene oxide dose", "Graphene oxide dose units", "Organism/Biological system", "score", "Source", "Passage"],
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)


In [ ]:
#query = "What adverse molecular or cellular events attributable to graphene #oxide diameter are described in these documents?"
#docs = docsearch.as_retriever().get_relevant_documents(query)
#chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff", #prompt=doc_prompt)
#diameter = """
#Adverse outcome (What was the adverse outcome?):
#Bioassay (What specific bioassay was performed?):
#Material diameter measurement (What type of material diameter measurement is #reported?):
#Numerical value (What is the numerical value of the diameter measurement?):
#Units (What are the units of the diameter measurement, if applicable?):
#Graphene oxide dose (What was the graphene oxide dose that produced this #outcome?):
#Organism/Biological system (What organism or biological system was the assay #performed on (include strain, if applicable)?
#Which lines (a passage of the text) provided this answer?"""
#
#res = chain({"input_documents": docs, "question": diameter}, #return_only_outputs=False)
#res['output_text'].split("\n")

This approach does not seem to work -getting only one response.

Below a different approach:

In [ ]:

documents = os.listdir(paper_path)

results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    diameter = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Material diameter measurement (What type of material diameter measurement is    reported?):
    Numerical value (What is the numerical value of the diameter measurement?):
    Units (What are the units of the diameter measurement, if applicable?):
    Graphene oxide dose (What was the graphene oxide dose that produced this    outcome?):
    Organism/Biological system (What organism or biological system was the assay    performed on (include strain, if applicable)?
    Which lines (a passage of the text) provided this answer?"""
    res = chain({"input_documents": docs, "question": diameter},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
